In [2]:
import equinox as eqx
import jax

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [3]:
class Linear(eqx.Module):
    def __init__(self, in_features, out_features):
        self.in_features = in_features
        self.out_features = out_features
        self.weight = eqx.Parameter(jax.random.normal(jax.random.PRNGKey(0), (out_features, in_features)))
        self.bias = eqx.Parameter(jax.random.normal(jax.random.PRNGKey(0), (out_features,)))

    def forward(self, x):
        return jax.numpy.dot(self.weight, x) + self.bias

In [4]:
m = Linear(3, 4)
print(m)

AttributeError: Cannot set attribute in_features